# Scratch Code from Primary NN Models

### segment_signal method with time tracking

In [ ]:
# Used for vectorized input WITHOUT SQL pre-processing
    # ACTIVELY WORKING ON THIS METHOD FOR SPEED OPTIMIZATION (RIGHT NOW IT TAKES TOO LONG)
def segment_signal(data_inputs, data_full, input_window_size = input_window_size):
    TIME0 = time.time()
    if  model_architecture == 'FCN':
        segments = np.empty((0,input_window_size*num_channels + num_anthropometrics))
    elif model_architecture == 'CNN':
        segments = np.empty((0, input_window_size, num_channels + num_anthropometrics))
    labels = np.empty((0))
    
    TIME1 = time.time()
    TIME_INIT = TIME1 - TIME0
    time_tracker_full_loop = [0]
    time_tracker_cols = [0]
    time_tracker_segment_stack = [0]
    time_tracker_label_stack = [0]
    entered_stracking = [0]
    count = 0
    
    for (start, end) in windows(data_full['timestamp'], input_window_size):
        
        time_tracker_full_loop_start = time.time()
        #time_range_tracker = np.hstack([time_range_tracker, (time_tracker_full_loop[count+1] - time_tracker_full_loop[count]) ]) #count+
        count += 1
        time_tracker_cols_start = time.time()
        
        a = data_inputs["bounce"][start:end]
        b = data_inputs["braking"][start:end]
        c = data_inputs["cadence"][start:end]
        d = data_inputs["ground_contact"][start:end]
        e = data_inputs["pelvic_drop"][start:end]
        f = data_inputs["pelvic_rotation"][start:end]
        g = data_inputs["pelvic_tilt"][start:end]
        if model_architecture == 'FCN':
            aa = data_inputs["age"][start]
            bb = data_inputs["weight"][start]
            cc = data_inputs["height"][start]
            dd = data_inputs["gender"][start] 
        elif model_architecture == 'CNN':
            aa = data_inputs["age"][start:end]
            bb = data_inputs["weight"][start:end]
            cc = data_inputs["height"][start:end]
            dd = data_inputs["gender"][start:end]         
        
        time_tracker_cols_end = time.time()
        time_tracker_cols = np.hstack([time_tracker_cols, time_tracker_cols_end - time_tracker_cols_start])
        time_tracker_segment_stack_start = time.time()
        
        if(end < data_full.shape[0] and len(data_full['timestamp'][start:end]) == input_window_size and data_full['activity_id'][start]==data_full['activity_id'][end]):
            
            entered_stracking = np.hstack([entered_stracking, count])
            
            if model_architecture == 'FCN':
                segments_toadd = np.vstack([np.dstack([a,b,c,d,e,f,g])])
                segments_toadd_reshape = segments_toadd.reshape(input_window_size * num_channels)
                segments = np.vstack([segments,np.hstack([aa,bb,cc,dd,segments_toadd_reshape])])
            elif model_architecture == 'CNN':
                segments = np.vstack([segments,np.dstack([aa,bb,cc,dd,a,b,c,d,e,f,g])])
            
            time_tracker_segment_stack_end = time.time()
            time_tracker_segment_stack = np.hstack([time_tracker_segment_stack, time_tracker_segment_stack_end - time_tracker_segment_stack_start])
            time_tracker_label_stack_start = time.time()
            
            start_labeling = np.int(np.floor(start+(end-start)/2) - np.floor(label_window_size/2))
            end_labeling = start_labeling + label_window_size
            if speed_bucket_size == '0.1':
                labels = np.append(labels,np.around(np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=1)) # round to nearest decimal
            elif speed_bucket_size == '0.5':
                labels = np.append(labels,np.around(2*np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=0)/2) # round to nearest half unit
            elif speed_bucket_size == 'none_use_regression':
                labels = np.append(labels,np.mean(data_full["gps_speed_true"][start_labeling:end_labeling])) # no rounding, use regression
            
            time_tracker_segment_label_end = time.time()
            time_tracker_label_stack = np.hstack([time_tracker_label_stack, time_tracker_segment_label_end - time_tracker_label_stack_start])
            
        else:
            entered_stracking = np.hstack([entered_stracking, 0])
        time_tracker_full_loop_end = time.time()
        time_tracker_full_loop = np.hstack([time_tracker_full_loop, time_tracker_full_loop_end - time_tracker_full_loop_start])
        
    TIME2 = time.time()
    TIME_LOOP = TIME2 - TIME1
    print(TIME_INIT)
    print(TIME_LOOP)
    print(time_tracker_full_loop[0:1000])
    print(time_tracker_full_loop[10000:11000])
    print(time_tracker_cols[0:1000])
    print(time_tracker_cols[10000:11000])
    print(time_tracker_segment_stack[0:1000])
    print(time_tracker_segment_stack[10000:11000])
    print(time_tracker_label_stack[0:1000])
    print(time_tracker_label_stack[10000:11000])
    
    print(entered_stracking)
    return segments, labels

### Debugger code for normalizing input dataset

In [ ]:
# Debugging Helper Code

# #dataset = read_data(myFileLocation)
# dataset_full = dataset
# print(dataset[1:4])
# print(dataset_full[1:4])
# #dataset['gender':'pelvic_tilt_lag_0'] = (dataset - dataset.mean())/dataset.std()
# dataset_inputs = dataset.loc[:, 'gender':'pelvic_tilt_lag_0'] # normalize all columns from gender to pelvic_tilt_lag_0
# dataset_inputs_normalized = (dataset_inputs - dataset_inputs.mean())/dataset_inputs.std()
# dataset_full.drop(["gender":"pelvic_tilt_lag_0"])
# dataset_full = [dataset_full, dataset_inputs_normalized]
# print(dataset[1:4])
# print(dataset_full[1:4])

### Debuger code for saving segment and labels as NP arrays

In [ ]:
# Debugging helper code

#     if data_input_table_structure == 'Raw_Timeseries':
#         segments, labels = segment_signal(dataset_inputs_normalized, dataset)
#     elif data_input_table_structure == 'Vectorized_By_Row':
#         segments, labels = segment_signal_FCN_vector(dataset_inputs_normalized, dataset)

#     if speed_bucket_size != 'none_use_regression': # if not using regression, convert to one-hot vector labels
#          labels_to_number = np.unique(labels) # Caches "labels_to_number" in order to use in rmse calculation for classification
#          labels = np.asarray(pd.get_dummies(labels), dtype = np.int8) # one-hot labels to classify nearest bucket
#          num_buckets_total = len(labels[1]) # total number of classification buckets that exist in the dataset (here, classification bucket == classification class)
            

            
# segments2 = np.load('../Saved NP Arrays/TESTPICKLE2.npy', allow_pickle=True)
# np.save('../Saved NP Arrays/TESTPICKLE2', segments, allow_pickle=True)
# #save_pickle_obj(segments, 'TESTPICKLE')

print(segments.shape[0])
print(len(segments))
#reshaped_segments = segments.reshape(len(segments), 1,input_num_timestamps, num_channels)

print(segments[0,0,1:10])
print(labels.shape)
print(segments.shape)

### Variables removed from code for simplicity

In [ ]:
# Helper Variables for CNN model    # THIS IS SILLY. GET RID OF THESE TWO VARIABLES, ADD COMMENTS FOR EASE OF USE/READING INSTEAD
if model_architecture == 'CNN':
    num_rows_per_example = segments.shape[1] # same as input_window_size
    num_cols_per_example = segments.shape[2] # same as total channels

### Convolution accidentally 2D

In [ ]:
    model.add(Conv2D(num_filters, (kernel_size,1),input_shape=(input_window_size, num_channels+num_anthropometrics,1),activation=activation_conv_layer))
    model.add(MaxPooling2D(pool_size=(max_pool_kernel_size,1),padding='valid',strides=(2,1)))
    model.add(Conv2D(num_filters//10, (kernel_size,1),activation=activation_conv_layer)) # add additional CNN layer

### Train Dev Split mistakes due to using Conv 2D and switching to 1D

In [ ]:
train_dev_split = np.random.rand(len(segments)) < 0.90
if data_input_table_structure == 'Raw_Timeseries':
    #reshapedSegments = segments.reshape(segments.shape[0], input_window_size, num_channels+num_anthropometrics, 1)
        # Making a change to swith to Conv1D
    print(segments.shape)
    reshapedSegments = segments.reshape(segments.shape[0], input_window_size, num_channels+num_anthropometrics)
        # this line is not needed at all anymore, it does 
    print(reshapedSegments.shape)
    X_train = reshapedSegments[train_dev_split]
    X_test = reshapedSegments[~train_dev_split]
elif data_input_table_structure == 'Vectorized_By_Row':
    X_train = segments[train_dev_split]
    X_test = segments[~train_dev_split]
y_train = labels[train_dev_split]
y_test = labels[~train_dev_split]

### Code snippets from before Concatenated anthropometrics in the CNN

In [ ]:

# Used for vectorized input WITHOUT SQL pre-processing
def segment_signal(data_inputs, data_full, input_window_size = input_window_size):
    if  model_architecture == 'FCN':
        segments = np.empty((0,input_window_size*num_channels + num_anthropometrics))
    elif model_architecture == 'CNN':
        segments = np.empty((0, input_window_size, num_channels + num_anthropometrics))
    labels = np.empty((0))
    for (start, end) in windows(data_full['timestamp'], input_window_size):
        a = data_inputs["bounce"][start:end]
        b = data_inputs["braking"][start:end]
        c = data_inputs["cadence"][start:end]
        d = data_inputs["ground_contact"][start:end]
        e = data_inputs["pelvic_drop"][start:end]
        f = data_inputs["pelvic_rotation"][start:end]
        #g = data_inputs["pelvic_tilt"][start:end]
        if model_architecture == 'FCN':
            aa = data_inputs["age"][start]
            bb = data_inputs["weight"][start]
            cc = data_inputs["height"][start]
            dd = data_inputs["gender"][start] 
        elif model_architecture == 'CNN':
            aa = data_inputs["age"][start:end]
            bb = data_inputs["weight"][start:end]
            cc = data_inputs["height"][start:end]
            dd = data_inputs["gender"][start:end] 
        if(end < data_full.shape[0] and len(data_full['timestamp'][start:end]) == input_window_size and data_full['activity_id'][start]==data_full['activity_id'][end]):
            if model_architecture == 'FCN':
                segments_toadd = np.vstack([np.dstack([a,b,c,d,e,f,g])])
                segments_toadd_reshape = segments_toadd.reshape(input_window_size * num_channels)
                segments = np.vstack([segments,np.hstack([aa,bb,cc,dd,segments_toadd_reshape])])
            elif model_architecture == 'CNN':
                #segments = np.vstack([segments,np.dstack([aa,bb,cc,dd,a,b,c,d,e,f,g])])
                segments = np.vstack([segments,np.dstack([aa,bb,cc,dd,a,b,c,d,e,f])])
            start_labeling = np.int(np.floor(start+(end-start)/2) - np.floor(label_window_size/2))
            end_labeling = start_labeling + label_window_size
            if speed_bucket_size == '0.1':
                labels = np.append(labels,np.around(np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=1)) # round to nearest decimal
            elif speed_bucket_size == '0.5':
                labels = np.append(labels,np.around(2*np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=0)/2) # round to nearest half unit
            elif speed_bucket_size == 'none_use_regression':
                labels = np.append(labels,np.mean(data_full["gps_speed_true"][start_labeling:end_labeling])) # no rounding, use regression
    return segments, labels

# Used for vectorized input WITHOUT SQL pre-processing
def segment_signal_label_only(data_inputs, data_full, input_window_size = input_window_size):
    labels = np.empty((0))
    for (start, end) in windows(data_full['timestamp'], input_window_size):        
        if(end < data_full.shape[0] and len(data_full['timestamp'][start:end]) == input_window_size and data_full['activity_id'][start]==data_full['activity_id'][end]):
            start_labeling = np.int(np.floor(start+(end-start)/2) - np.floor(label_window_size/2))
            end_labeling = start_labeling + label_window_size
            if speed_bucket_size == '0.1':
                labels = np.append(labels,np.around(np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=1)) # round to nearest decimal
            elif speed_bucket_size == 'none_use_regression':
                labels = np.append(labels,np.mean(data_full["gps_speed_true"][start_labeling:end_labeling])) # no rounding, use regression
    return labels

# Preprocess data to input into model

np_array_file_string_segment = "../Saved NP Arrays/" + str(myFileName) + "_" + str(input_window_size) + "_" + str(label_window_size) + "_" + str(sample_stride) + "_segment.npy"
np_array_file_string_label = "../Saved NP Arrays/" + str(myFileName) + "_" + str(input_window_size) + "_" + str(label_window_size) + "_" + str(sample_stride) + "_label.npy"
np_array_file_string_label2num = "../Saved NP Arrays/" + str(myFileName) + "_" + str(input_window_size) + "_" + str(label_window_size) + "_" + str(sample_stride) + "_label2num.npy"

if os.path.isfile(np_array_file_string_segment):   # if this file already exists, load the relevant saved np arrays
    segments = np.load(np_array_file_string_segment, allow_pickle=True)
    labels = np.load(np_array_file_string_label, allow_pickle=True)
    labels_to_number = np.load(np_array_file_string_label2num, allow_pickle=True)
else:    # if this file does not exist, run segment_signal method and create np arrays for future use
    if data_input_table_structure == 'Raw_Timeseries':
        segments, labels = segment_signal(dataset_inputs_normalized, dataset)
    elif data_input_table_structure == 'Vectorized_By_Row':
        segments, labels = segment_signal_FCN_vector(dataset_inputs_normalized, dataset)
    if speed_bucket_size != 'none_use_regression': # if not using regression, convert to one-hot vector labels
         labels_to_number = np.unique(labels) # Caches "labels_to_number" in order to use in rmse calculation for classification
         labels = np.asarray(pd.get_dummies(labels), dtype = np.int8) # one-hot labels to classify nearest bucket
    np.save(np_array_file_string_segment, segments, allow_pickle=True)
    np.save(np_array_file_string_label, labels, allow_pickle=True)
    np.save(np_array_file_string_label2num, labels_to_number, allow_pickle=True)

    
def cnnModel():
    model = Sequential()
    model.add(Conv1D(num_filters, (kernel_size),input_shape=(input_window_size, num_channels+num_anthropometrics),activation=activation_conv_layer))
    model.add(MaxPooling1D(pool_size=(max_pool_kernel_size),padding='valid',strides=(2)))
    model.add(Conv1D(num_filters//10, (kernel_size),activation=activation_conv_layer)) # add additional CNN layer
    # model.add(Dropout(dropOutRatio)) # not used in our model # adding a dropout layer for the regularization
    model.add(Flatten()) # flatten the output in order to apply the fully connected layer
    model.add(Dense(num_hidden_units_fc_layers_CNN[0], activation=activations_fc_layers_CNN[0])) # add first fully connected layer
    # model.add(BatchNormalization())
    # model.add(Dropout(dropout_rate_fc_layers_CNN[0]))
    # Intermediate fully connected layerslayers
    for L in range(1, num_hidden_fc_layers_CNN):
        model.add(Dense(num_hidden_units_fc_layers_CNN[L], activation=activations_fc_layers_CNN[L]))
        # model.add(BatchNormalization())
        model.add(Dropout(dropout_rate_fc_layers_CNN[L]))
    # Last hidden layer
    if speed_bucket_size != 'none_use_regression': # if classification, use softmax for last layer
        model.add(Dense(num_buckets_total, activation='softmax'))
    else:                                          # if regression, use linear for last layer
        model.add(Dense(1,activation='linear'))
    return model

### Remove 0.5 speed bucket option (no longer used)

In [ ]:
# In segment signal code

#raw timeseries
    elif speed_bucket_size == '0.5':
        labels = np.append(labels,np.around(2*np.mean(data_full["gps_speed_true"][start_labeling:end_labeling]),decimals=0)/2) # round to nearest half unit
#vectorized        
    elif speed_bucket_size == '0.5':
        labels = np.around(2*np.mean(labels_before_avg, axis=1),decimals=0)/2
    